
### Investigate Database Structures

In [0]:
%sql

SHOW DATABASES;

In [0]:
%sql

SELECT CURRENT_DATABASE();

In [0]:
%sql

USE default;

In [0]:
%sql

SHOW TABLES;

In [0]:
%sql

DESCRIBE EXTENDED caden_os_dev.os_silver_dev.users_silver_dev;

In [0]:
%sql

select * from caden_os_dev.os_silver_dev.users_silver_dev;

In [0]:
%sql

DESCRIBE EXTENDED caden_os_dev.os_silver_dev.transactions_silver_dev;

In [0]:
%sql

select * from caden_os_dev.os_silver_dev.transactions_silver_dev;

In [0]:
%sql

select t_transaction_category, count(t_caden_alias)
from caden_os_dev.os_silver_dev.transactions_silver_dev
group by 1

Databricks visualization. Run in Databricks to view.


## Create Tables 


### Create Mananged Table From Spark Dataframe Using pySpark (Method 1)

In [0]:
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")

In [0]:
df = spark.sql(""" select * from caden_os_dev.os_silver_dev.transactions_silver_dev limit 100 """)

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("default.train_ling_test")

In [0]:
%sql

select * from default.train_ling_test;


### Create Managed Table From Existing Table Using SQL (Method 2)

In [0]:
%sql

CREATE OR REPLACE TABLE default.train_ling_test_2 AS
SELECT * 
FROM default.train_ling_test
WHERE t_transaction_category = "Transfer";

In [0]:
%sql

SELECT * 
FROM default.train_ling_test_2;


### Create Managed Table From Spark Dataframe Using SQL (Method 3)

In [0]:
df.createOrReplaceTempView('df')

In [0]:
%sql

CREATE OR REPLACE TABLE default.train_ling_test_3 AS
SELECT * 
FROM df
WHERE t_transaction_category = "Transfer";



### Create Unmananged Table From Spark Dataframe Using pySpark (Method 4)

In [0]:
(df.write.format("delta")
.mode("overwrite")
.option("path", "/user/hive/warehouse").saveAsTable("train_ling_test_4"))


### Create Unmanaged Table From Spark Dataframe Using SQL (Method 5)

In [0]:
df.createOrReplaceTempView("df")

In [0]:
%sql

CREATE OR REPLACE TABLE default.train_ling_test_5 
USING delta
LOCATION "/user/hive/warehouse/train_ling_test_5/"
AS
SELECT * 
FROM df
WHERE t_transaction_category = "Transfer";


## Drop Tables

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/train_ling_test', True)

In [0]:
for i in range(2, 6):
    dbutils.fs.rm(f'dbfs:/user/hive/warehouse/train_ling_test_{i}', True)

In [0]:
display(dbutils.fs.ls('dbfs:/user/hive/warehouse/'))

In [0]:
spark.sql("DROP TABLE hive_metastore.default.train_ling_test")

In [0]:
for i in range(2, 6):
    spark.sql(f"DROP TABLE hive_metastore.default.train_ling_test_{i}")